#### Importing all Required Libraries

In [ ]:
import os
import anthropic
import json
from collections import OrderedDict
import time
from scrapper import scrape_website

# Set the environment variable within the notebook
os.environ["ANTHROPIC_API_KEY"] = "you-api-key"

#### Data Scrapping from Website

In [2]:
# Scrapping the Given Website
url = "https://www.opentable.com/r/metropolitan-grill-seattle"
name_of_file = scrape_website(url)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x0083EC13+23731]
	(No symbol) [0x007CC394]
	(No symbol) [0x006ABE63]
	(No symbol) [0x0068D92B]
	(No symbol) [0x00717F7F]
	(No symbol) [0x0072AD99]
	(No symbol) [0x00711BF6]
	(No symbol) [0x006E3F35]
	(No symbol) [0x006E4EBD]
	GetHandleVerifier [0x00B1F0D3+3039603]
	GetHandleVerifier [0x00B32DEA+3120778]
	GetHandleVerifier [0x00B2B592+3089970]
	GetHandleVerifier [0x008D43B0+635984]
	(No symbol) [0x007D4DCD]
	(No symbol) [0x007D2068]
	(No symbol) [0x007D2205]
	(No symbol) [0x007C4FD0]
	BaseThreadInitThunk [0x767F7BA9+25]
	RtlInitializeExceptionChain [0x77DEC0CB+107]
	RtlClearBits [0x77DEC04F+191]


#### Prompt Engineering

In [ ]:
def classify_review(client : anthropic.Anthropic,review : str, model = "claude-3-5-sonnet-20241022")->list:
    """
    Classify a review by extracting food and service-related text.

    Parameters:
        client (anthropic.Anthropic): A pre-initialized Anthropic client object.
        text (str): The input review text to classify.
        model (str): The Anthropic model to use. Default is 'claude-3-5-sonnet-20241022'.

    Returns:
        str: The extracted text related to food and service.
    """
    # review = preprocess_text(review)
    message = client.messages.create(
        model= model,
        max_tokens=1000,
        temperature=0,
        system= """<userStyle>Normal</userStyle>
                    <longContext> 
                        [NO_HALLUCINATION][PROTECT_PRIVACY]	You are an AI assistant trained to categorize user feedback into predefined categories,you should categorize the text into three sections:
                        1. **Food**: Extract and return text referring to the quality of the food. 
                        2. **Service/Staff**: Extract and return text referring to the quality of the service or the staff. 
                        3. **Other Category**: Any part of the review that doesn't relate to food or service, such as ambiance, pricing, or atmosphere. 
                        Make sure **Food** and **Service** are distinct and don't overlap. If a part of the review refers to both, categorize accordingly. 
                        If no part is identified for a specific category, respond with "None" for that category.

                        If you encounter potentially identifying information (e.g., specific names, locations, phone numbers,emails, or personal details), skip it and do not produce it in the output.

                        Avoid generating any content that isn't directly supported by the review text. If no relevant information can be derived from the review, provide empty fields or "None" accordingly.

                        Strictly follow the output format provided in the examples, dont add extra lines, not even greetings. It should be a 3 lines output with the given categories.

                    </longContext>
                    <example>
                        Example 1:
                            Text: Best steakhouse experience we've had in a long time, the service was exceptional! Truly made our date night marvelous.
                            Response:  
                                Food: None
                                Service: the service was exceptional!
                                Other: None
                                
                        Example 2:
                            Text: Spectacular customer service, great atmosphere and even better food..
                            Response:
                                Food: even better food
                                Service: Spectacular customer service
                                Other: great atmosphere
                                
                        Example 3:
                            Text: Old school steakhouse vibe with very attentive service. Food was delicious. The steak was perfectly cooked to our specifications. Remember steakhouses are not inexpensive! But it was worth it..
                            Response:
                                Food: Food was delicious. The steak was perfectly cooked to our specifications.
                                Service: Old school steakhouse vibe with very attentive service
                                Other: Remember steakhouses are not inexpensive! But it was worth it..
                    </example>
                    </longContext>
    """,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": review
                    }
                ]
            }
        ]
    )
    lines = (message.content[0].text).splitlines()
    return lines

def validate_response(review: list)->list:
    if(len(review) == 3):
        return review
    elif(len(review) > 3):
        new_list = []
        for i in review:
            if "Food: " in i or "Service: " in i or "Other: " in i:         # Keeping Only the three tags in case the response generated has more than 3 lines
                new_list.append(i)
                if len(new_list) == 3:
                    return new_list

def classify_review_with_retry(client, review_content, retries=5, base_delay=1.2):
    """
    Call the classify_review function with retry logic.
    """
    for attempt in range(retries):
        try:
            return classify_review(client, review_content)  # Your API call
        except Exception as e:
            if attempt < retries - 1:  # Retry if attempts are left
                delay = base_delay * (2 ** attempt)  # Exponential backoff
                print(f"API call failed: {e}. Retrying in {delay:.2f} seconds...")
                time.sleep(delay)
            else:
                print(f"API call failed after {retries} retries: {e}")
                raise  # Re-raise the exception after final failure

with open('reviews.json', 'r') as file:
    reviews_dict = OrderedDict(json.load(file))

In [ ]:

client = anthropic.Anthropic()
classified_response = OrderedDict()
n = 1500                                                # Number of reviews to analyze
for customer,review in reviews_dict.items():
    response = classify_review_with_retry(client,review['Content'])
    response = validate_response(response)
    try:
        food = response[0].split(":", 1)[1]
    except Exception as e:
        food = None
    try:
        service = response[1].split(":", 1)[1]
    except Exception as e:
        service = None
    try:
        other = response[2].split(":", 1)[1]
    except Exception as e:
        other = None
    category = {}
    category["Food"] = food.strip()
    category["Service"] = service.strip()
    category["Other"] = other.strip()
    classified_response[customer] = category
    n -= 1
    if n == 0:
        break


API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}. Retrying in 1.20 seconds...
API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}. Retrying in 1.20 seconds...
API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}. Retrying in 1.20 seconds...
API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}. Retrying in 1.20 seconds...
API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}. Retrying in 1.20 seconds...
API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}. Retrying in 1.20 seconds...
API call failed: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Inte

KeyboardInterrupt: 

In [ ]:
# Saving the results in the file

file_path = "classified_reviews.json"

with open(file_path, "w") as json_file:
    json.dump(classified_response, json_file, indent=4)

print(f"All reviews Saved to {file_path}")

### Merging the Classified Json with Original Json

In [ ]:

with open('reviews.json', 'r') as file:
    r_reviews = OrderedDict(json.load(file))

with open('classified_reviews.json', 'r') as file:
    classified_reviews = OrderedDict(json.load(file))

merged_reviews = OrderedDict()
for t_customer,t_review in classified_reviews.items():
    merged_reviews[t_customer] = r_reviews[t_customer]
    merged_reviews[t_customer]["Food"] = t_review.get("Food", "None")
    merged_reviews[t_customer]["Service"] = t_review.get("Service", "None")
    merged_reviews[t_customer]["Other"] = t_review.get("Other", "None")

file_path = "merged_reviews.json"

with open(file_path, "w") as json_file:
    json.dump(merged_reviews, json_file, indent=4)

print(f"All reviews Saved to {file_path}")

### GUI Development and Competitor Analysis

In [1]:
!streamlit run Homepage.py

^C
